In [1]:
# Injury Detection and Severity Pipeline (Enhanced)

import os
import cv2
import numpy as np
import torch
import torchvision
from torchvision import transforms, datasets
from torchvision.models import efficientnet_b0, densenet121
from torch import nn, optim
from torch.utils.data import DataLoader, random_split
from PIL import Image
from sklearn.utils.class_weight import compute_class_weight
import matplotlib.pyplot as plt

# ========== Step 1: Data Transformations ==========
train_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(20),
    transforms.ColorJitter(brightness=0.3, contrast=0.3, saturation=0.3),
    transforms.RandomPerspective(distortion_scale=0.2, p=0.5),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225])
])

val_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225])
])

# ========== Step 2: Dataset and Dataloader ==========
data_dir = "E:\D\college code\machinelearning\cv\Wound_dataset"
dataset = datasets.ImageFolder(data_dir, transform=train_transform)
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])
val_dataset.dataset.transform = val_transform

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=16, shuffle=False)

# ========== Step 3: Model Definitions ==========
def get_efficientnet_model(num_classes):
    model = efficientnet_b0(pretrained=True)
    model.classifier[1] = nn.Linear(model.classifier[1].in_features, num_classes)
    return model

def get_densenet_model(num_classes):
    model = densenet121(pretrained=True)
    model.classifier = nn.Linear(model.classifier.in_features, num_classes)
    return model

# ========== Step 4: Training Loop ==========
def train_model(model, criterion, optimizer, scheduler, train_loader, val_loader, num_epochs=10):
    best_acc = 0.0
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        running_corrects = 0

        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item() * inputs.size(0)
            running_corrects += torch.sum(preds == labels.data)

        scheduler.step()
        epoch_loss = running_loss / len(train_loader.dataset)
        epoch_acc = running_corrects.double() / len(train_loader.dataset)

        model.eval()
        val_corrects = 0
        with torch.no_grad():
            for inputs, labels in val_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                _, preds = torch.max(outputs, 1)
                val_corrects += torch.sum(preds == labels.data)

        val_acc = val_corrects.double() / len(val_loader.dataset)
        print(f"Epoch {epoch+1}/{num_epochs}, Loss: {epoch_loss:.4f}, Train Acc: {epoch_acc:.4f}, Val Acc: {val_acc:.4f}")

        if val_acc > best_acc:
            best_acc = val_acc
            torch.save(model.state_dict(), 'best_model.pth')

# ========== Step 5: Severity Estimation ==========
def analyze_injury(image_path):
    image = Image.open(image_path).convert("RGB")
    transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406],
                             [0.229, 0.224, 0.225])
    ])
    image_tensor = transform(image).unsqueeze(0).to(device)

    model.eval()
    with torch.no_grad():
        output = model(image_tensor)
        _, pred = torch.max(output, 1)
        confidence = torch.nn.functional.softmax(output, dim=1)[0][pred].item()

    np_image = np.array(image.resize((224, 224)))
    gray = cv2.cvtColor(np_image, cv2.COLOR_RGB2GRAY)
    blur = cv2.GaussianBlur(gray, (5, 5), 0)
    _, thresh = cv2.threshold(blur, 60, 255, cv2.THRESH_BINARY_INV)
    contours, _ = cv2.findContours(thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

    max_contour = max(contours, key=cv2.contourArea) if contours else None
    area = cv2.contourArea(max_contour) if max_contour is not None else 0
    depth_estimate = min(1.0, area / (224 * 224)) * 5  # Simulated depth scale (0-5 cm)

    print(f"Predicted Class: {dataset.classes[pred]}\nConfidence: {confidence:.2f}\nArea: {area:.2f}px\nEstimated Depth: {depth_estimate:.2f} cm")

# ========== Step 6: Camera Integration ==========
def capture_from_camera():
    cap = cv2.VideoCapture(0)
    print("Press 's' to capture and analyze the injury.")

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        cv2.imshow("Live Camera", frame)
        if cv2.waitKey(1) & 0xFF == ord('s'):
            image_path = "captured_injury.jpg"
            cv2.imwrite(image_path, frame)
            analyze_injury(image_path)
            break

    cap.release()
    cv2.destroyAllWindows()

# ========== Main Execution ==========
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
num_classes = len(dataset.classes)

model = get_efficientnet_model(num_classes).to(device)  # OR use get_densenet_model

# Compute class weights
all_labels = [label for _, label in dataset.samples]
class_weights = compute_class_weight('balanced', classes=np.unique(all_labels), y=all_labels)
class_weights = torch.FloatTensor(class_weights).to(device)

criterion = nn.CrossEntropyLoss(weight=class_weights)
optimizer = optim.Adam(model.parameters(), lr=0.001)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.1)

if __name__ == '__main__':
    print("Training model...")
    train_model(model, criterion, optimizer, scheduler, train_loader, val_loader)
    print("Training complete. Launching camera...")
    capture_from_camera()

<>:36: SyntaxWarning: invalid escape sequence '\D'
<>:36: SyntaxWarning: invalid escape sequence '\D'
C:\Users\menar\AppData\Local\Temp\ipykernel_1424\424881047.py:36: SyntaxWarning: invalid escape sequence '\D'
  data_dir = "E:\D\college code\machinelearning\cv\Wound_dataset"
c:\Python312\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Python312\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=EfficientNet_B0_Weights.IMAGENET1K_V1`. You can also use `weights=EfficientNet_B0_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Training model...
Epoch 1/10, Loss: 1.4914, Train Acc: 0.4330, Val Acc: 0.7273
Epoch 2/10, Loss: 0.5147, Train Acc: 0.8148, Val Acc: 0.7386
Epoch 3/10, Loss: 0.5362, Train Acc: 0.8405, Val Acc: 0.7159
Epoch 4/10, Loss: 0.4157, Train Acc: 0.8746, Val Acc: 0.6932
Epoch 5/10, Loss: 0.4731, Train Acc: 0.8860, Val Acc: 0.7386
Epoch 6/10, Loss: 0.2322, Train Acc: 0.9430, Val Acc: 0.8068
Epoch 7/10, Loss: 0.1322, Train Acc: 0.9658, Val Acc: 0.7955
Epoch 8/10, Loss: 0.1197, Train Acc: 0.9715, Val Acc: 0.8295
Epoch 9/10, Loss: 0.1108, Train Acc: 0.9715, Val Acc: 0.8295
Epoch 10/10, Loss: 0.1097, Train Acc: 0.9544, Val Acc: 0.8068
Training complete. Launching camera...
Press 's' to capture and analyze the injury.
Predicted Class: Stab_wound
Confidence: 0.39
Area: 15049.50px
Estimated Depth: 1.50 cm
